In [1]:
# load libraries
import nibabel as nib
import pandas as pd
import numpy as np

from nilearn.connectome import ConnectivityMeasure
from nilearn.maskers import NiftiLabelsMasker
from nilearn import datasets
from pathlib import Path

In [2]:
def correct_mean_var(tseries):
    
    '''
    Correct time series to zero mean and unit variance. Adapted from: 
    https://github.com/SIMEXP/niak/blob/master/commands/statistics/niak_correct_mean_var.m,
    but for a 1D array with the default 'mean_var' only.
    ------------------------------------- 
    TSERIES_N = CORRECT_MEAN_VAR(TSERIES)
    
    INPUT
    TSERIES: (1D array) time series.
    
    OUTPUT
    TSERIES_N: (1D array) same as data after mean/variance correction.
    '''
    
    nt = tseries.shape[0]
    tseries = tseries.reshape(nt,-1)
    
    mean_ts = np.mean(tseries,0)
    std_ts = tseries.std()
    
    tseries_n = (tseries - mean_ts)/std_ts
    tseries_n = np.ravel(tseries_n)
    
    return (tseries_n)

In [3]:
def build_size_roi(mask):
    
    '''
    Extract labels and size of ROIs given a mask. Adapted from:
    https://github.com/SIMEXP/niak/blob/master/commands/SI_processing/niak_build_size_roi.m
    ------------------------------------- 
    [SIZE_ROI,LABELS_ROI] = BUILD_SIZE_ROI(MASK)
    
    INPUT
    MASK: (array) voxels belonging to no region are coded with 0, those 
    belonging to region I are coded with I (I being a positive integer).
    
    OUTPUTS
    SIZE_ROI: (vector) SIZE_ROI(I) is the number of voxels in region number I.

    LABELS_ROI: (vector) LABELS_ROI(I) is the label of region I.
    '''
    
    nb_mask = len(mask)
    size_roi = np.zeros([nb_mask,1])

    labels_roi = np.unique(mask)
    labels_roi = labels_roi[labels_roi!=0]

    for num_r in range(nb_mask):
        size_roi[num_r] = np.count_nonzero(mask==labels_roi[num_r])
    
    return (size_roi, labels_roi)

In [4]:
def mat2lvec(mat):
    
    '''
    Convert a symmetric matrix into a vector (diagonal elements included). Adapted from:
    https://github.com/SIMEXP/niak/blob/master/commands/formats/niak_mat2lvec.m
    ------------------------------------- 
    LVEC = MAT2LVEC(MAT)
    
    INPUT
    MAT: (array) a square matrix. MAT should be symmetric. Diagonal elements 
    will be included.
    
    OUTPUTS
    LVEC: (vector) a vectorized version of mat. Low-triangular and diagonal values are kept.
    '''

    N = mat.shape[1]
    mask_l = np.tril(np.ones(N, dtype=int))
    mask_l = mask_l>0
    lvec = mat[mask_l]
    
    return (lvec)

In [5]:
# Load one subject of brain development fmri data from nilearn
data = datasets.fetch_development_fmri(n_subjects=1, reduce_confounds=True)
fmri_filenames = data.func[0]
reduced_confounds = data.confounds[0]  # This is a preselected set of confounds

In [6]:
# Load MIST 64 atlas as mask
parcellation = datasets.fetch_atlas_basc_multiscale_2015(version='sym')['scale064']
img = nib.load(parcellation)
mask = np.array(img.dataobj)
mask.shape

(53, 64, 52)

In [7]:
# Compute time series. Here, I am setting 'standardize' to False and then using the function correct_mean_var, 
# since the time series I will be using are not standardized so wanted to check functionality
masker = NiftiLabelsMasker(labels_img=img, standardize=False,
                           memory='nilearn_cache', verbose=5)
time_series = masker.fit_transform(fmri_filenames, confounds=reduced_confounds)

[NiftiLabelsMasker.fit_transform] loading data from Nifti1Image('/Users/natashaclarke/nilearn_data/basc_multiscale_2015/template_cambridge_basc_multiscale_nii_sym/template_cambridge_basc_multiscale_sym_scale064.nii.gz')
Resampling labels
[Memory]0.4s, 0.0min    : Loading _filter_and_extract...
__________________________________filter_and_extract cache loaded - 0.0s, 0.0min


In [8]:
# Correct time series to zero mean and unit variance
tseries_n = [correct_mean_var(ts) for ts in time_series.T]  
tseries_n = np.array(tseries_n).T

In [9]:
# Build connectome from time series
correlation_measure = ConnectivityMeasure(kind='correlation')
conn = correlation_measure.fit_transform([time_series])[0]

In [10]:
# Compute AFC. The following cells are adapted from https://github.com/SIMEXP/niak/blob/master/bricks/connectome/niak_brick_connectome.m#L248-L262 
# and I have added the corresponding matlab line

# NIAK: ir = var(tseries_n,[],1)';
ir = tseries_n.var(axis=0) 

In [11]:
# Get size of ROIs

# NIAK: N = niak_build_size_roi (all_mask{num_m});
N, labels = build_size_roi(mask) 
N

array([[1033.],
       [ 420.],
       [ 360.],
       [ 805.],
       [ 517.],
       [ 747.],
       [ 404.],
       [ 285.],
       [ 753.],
       [ 372.],
       [ 977.],
       [1133.],
       [ 702.],
       [ 913.],
       [ 816.],
       [ 426.],
       [ 638.],
       [ 769.],
       [ 776.],
       [ 724.],
       [ 356.],
       [ 947.],
       [ 779.],
       [ 936.],
       [1072.],
       [ 725.],
       [1023.],
       [ 952.],
       [1132.],
       [ 685.],
       [ 656.],
       [ 751.],
       [ 377.],
       [ 644.],
       [1015.],
       [1013.],
       [ 868.],
       [ 722.],
       [ 777.],
       [ 951.],
       [1126.],
       [ 646.],
       [1052.],
       [ 940.],
       [1487.],
       [1749.],
       [1255.],
       [ 966.],
       [ 708.],
       [1105.],
       [1317.],
       [1173.],
       [1206.]])

In [12]:
# NIAK: mask_0 = (N==0)|(N==1);
mask_0 = (N==0)|(N==1) 

In [13]:
# NIAK: ir = ((N.^2).*ir-N)./(N.*(N-1));
# This is not correct because it should result in a 64x1 array, but I can't work out what it should be...
ir = ((N*N)*ir-N)/(N*(N-1))
ir

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [14]:
# NIAK: ir(mask_0) = 0;
# throws error
ir[mask_0] = 0
ir[mask_0]

IndexError: boolean index did not match indexed array along dimension 1; dimension is 64 but corresponding boolean dimension is 1

In [15]:
# NIAK: conn(eye(size(conn))>0) = ir; % A tricky formula to add the average correlation within each network, at the voxel level, on the diagonal
# I think the below is doing what the NIAK code is, but unsure since the result of my 'ir' is incorrect
diagonal = np.diag_indices_from(conn)
conn[diagonal] = ir

ValueError: shape mismatch: value array of shape (53,64)  could not be broadcast to indexing result of shape (64,)

In [16]:
# Convert conn to lower vector
conn_lvec = mat2lvec(conn)

In [17]:
# Fisher-Z transform, unsure how to handle the inf and nan values?
conn_z = np.arctanh(conn_lvec) 

<ipython-input-17-a37c8a662f89>:2: RuntimeWarning: divide by zero encountered in arctanh
  conn_z = np.arctanh(conn_lvec)
